In [3]:
# 7.2.3 - 의미적 유사성 기반 예시 선택자 (SemanticSimilarityExampleSelector)
# 입력과 의미적으로 가장 유사한 예시를 자동으로 선택하는 고급 기법

# 필요한 라이브러리 설치
# !pip install -U langchain==0.2.17 langchain-openai langchain-community
# !pip install "deeplake[enterprise]<4.0.0"

# =============================================================================
# API 키 설정 (독자용)
# =============================================================================
import os
import getpass

print("필요한 API 키들을 설정해주세요:")

# OpenAI API 키 설정
if 'OPENAI_API_KEY' not in os.environ:
    openai_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    if openai_key:
        os.environ['OPENAI_API_KEY'] = openai_key
        print("✅ OpenAI API 키가 설정되었습니다!")
else:
    print("✅ 기존 OpenAI API 키를 사용합니다.")

# Activeloop (DeepLake) 토큰 설정 (선택사항)
if 'ACTIVELOOP_TOKEN' not in os.environ:
    print("\n📝 Activeloop 토큰은 선택사항입니다 (로컬에서만 실행할 경우 생략 가능)")
    activeloop_token = getpass.getpass("Activeloop 토큰 (선택사항, 엔터로 건너뛰기): ")
    if activeloop_token:
        os.environ['ACTIVELOOP_TOKEN'] = activeloop_token
        print("✅ Activeloop 토큰이 설정되었습니다!")
    else:
        print("⏭️ Activeloop 토큰을 건너뛰었습니다 (로컬 저장소 사용)")

print("\n" + "=" * 80)

# =============================================================================
# 라이브러리 import
# =============================================================================
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import DeepLake

print("7.2.3 의미적 유사성 기반 예시 선택자")
print("=" * 80)

print("""
원서 코드의 주요 수정사항:
1. 라이브러리 버전 호환성 문제 해결
2. SemanticSimilarityExampleSelector 생성 방식 변경
3. vectorstore 객체 직접 전달 방식으로 변경
4. 예시 삽입 방법 개선

핵심 개념 - SemanticSimilarityExampleSelector:
- 입력과 의미적으로 가장 유사한 예시를 자동 선택
- 벡터 임베딩을 사용한 유사도 계산
- 대량의 예시 중에서 효율적으로 관련성 높은 예시만 선택
""")

# =============================================================================
# 예시 프롬프트 템플릿 및 데이터
# =============================================================================
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

# 온도 변환 예시 데이터
examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"}, 
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

print(f"사용할 예시 개수: {len(examples)}")
for i, ex in enumerate(examples, 1):
    print(f"{i}. {ex['input']} → {ex['output']}")

# =============================================================================
# 벡터 임베딩 및 벡터스토어 설정
# =============================================================================
print("\n단계 1: 벡터 임베딩 및 벡터스토어 설정")

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

# DeepLake 벡터스토어 생성 (로컬)
dataset_path = "./deeplake_temperature"
vectorstore = DeepLake(dataset_path=dataset_path, embedding_function=embedding)

print("✅ DeepLake 벡터스토어가 생성되었습니다")

# =============================================================================
# SemanticSimilarityExampleSelector 생성
# =============================================================================
print("\n단계 2: SemanticSimilarityExampleSelector 생성")

# 의미적 유사성 기반 예시 선택자 생성
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=1,  # 가장 유사한 1개 예시만 선택
    input_keys=["temperature"]  # 비교할 입력 키
)

print("✅ SemanticSimilarityExampleSelector가 생성되었습니다")

# =============================================================================
# 예시들을 벡터스토어에 삽입
# =============================================================================
print("\n단계 3: 예시들을 벡터스토어에 삽입")

try:
    # 예시들을 벡터스토어에 추가
    texts = []
    metadatas = []
    
    for ex in examples:
        text = f"Input: {ex['input']}\nOutput: {ex['output']}"
        texts.append(text)
        metadatas.append({"input": ex['input'], "output": ex['output']})
    
    vectorstore.add_texts(texts=texts, metadatas=metadatas)
    print(f"✅ {len(examples)}개의 예시가 벡터스토어에 추가되었습니다")
    
except Exception as e:
    print(f"⚠️ 벡터스토어 삽입 중 오류: {e}")
    print("💡 API 키가 올바르게 설정되었는지 확인하세요")

# =============================================================================
# FewShotPromptTemplate 구성
# =============================================================================
print("\n단계 4: FewShotPromptTemplate 구성")

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Convert the temperature from Celsius to Fahrenheit.\n(섭씨에서 화씨로 온도를 변환해줘.)",
    suffix="Input: {temperature}\nOutput:",
    input_variables=["temperature"],
    example_separator="\n\n"
)

print("✅ FewShotPromptTemplate이 구성되었습니다")

# =============================================================================
# 의미적 유사성 기반 예시 선택 테스트
# =============================================================================
print("\n" + "=" * 60)
print("의미적 유사성 기반 예시 선택 테스트")
print("=" * 60)

test_temperatures = ["5°C", "15°C", "25°C", "35°C", "45°C"]

try:
    for temp in test_temperatures:
        print(f"\n🌡️ 테스트 온도: {temp}")
        formatted_prompt = prompt.format(temperature=temp)
        print("선택된 프롬프트:")
        print(formatted_prompt)
        print("-" * 40)
        
except Exception as e:
    print(f"⚠️ 테스트 중 오류: {e}")

# =============================================================================
# 동적 예시 추가 및 재테스트
# =============================================================================
print("\n" + "=" * 60)
print("동적 예시 추가 및 재테스트")
print("=" * 60)

print("새로운 예시 추가: 50°C → 122°F")

try:
    # 새로운 예시 추가
    new_text = "Input: 50°C\nOutput: 122°F"
    new_metadata = {"input": "50°C", "output": "122°F"}
    vectorstore.add_texts([new_text], [new_metadata])
    
    print("✅ 새로운 예시가 추가되었습니다")
    
    # 45°C로 다시 테스트 (50°C와 가장 유사할 것으로 예상)
    print(f"\n재테스트: 45°C (새로 추가된 50°C 예시와 가장 유사할 것으로 예상)")
    retest_prompt = prompt.format(temperature="45°C")
    print("선택된 프롬프트:")
    print(retest_prompt)
    
except Exception as e:
    print(f"⚠️ 동적 추가 테스트 중 오류: {e}")

# =============================================================================
# 다른 도메인 예제: 감정 분류
# =============================================================================
print("\n" + "=" * 60)
print("다른 도메인 예제: 감정 분류")
print("=" * 60)

# 감정 분류 예시
emotion_examples = [
    {"input": "I love this movie!", "output": "positive"},
    {"input": "This is terrible.", "output": "negative"},
    {"input": "It's okay, I guess.", "output": "neutral"},
    {"input": "Amazing performance!", "output": "positive"},
    {"input": "I hate waiting in line.", "output": "negative"},
    {"input": "The weather is fine.", "output": "neutral"}
]

# 감정 분류용 벡터스토어
emotion_dataset_path = "./deeplake_emotion"
emotion_vectorstore = DeepLake(dataset_path=emotion_dataset_path, embedding_function=embedding)

emotion_example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Text: {input}\nEmotion: {output}"
)

try:
    # 감정 예시들 추가
    emotion_texts = []
    emotion_metadatas = []
    
    for ex in emotion_examples:
        text = f"Text: {ex['input']}\nEmotion: {ex['output']}"
        emotion_texts.append(text)
        emotion_metadatas.append({"input": ex['input'], "output": ex['output']})
    
    emotion_vectorstore.add_texts(texts=emotion_texts, metadatas=emotion_metadatas)
    
    # 감정 분류용 선택자
    emotion_selector = SemanticSimilarityExampleSelector(
        vectorstore=emotion_vectorstore,
        k=2,  # 가장 유사한 2개 예시 선택
        input_keys=["text"]
    )
    
    # 감정 분류 프롬프트
    emotion_prompt = FewShotPromptTemplate(
        example_selector=emotion_selector,
        example_prompt=emotion_example_prompt,
        prefix="Classify the emotion of the given text as positive, negative, or neutral.\n(주어진 텍스트의 감정을 긍정, 부정, 중립으로 분류하세요.)",
        suffix="Text: {text}\nEmotion:",
        input_variables=["text"],
        example_separator="\n\n"
    )
    
    # 감정 분류 테스트
    emotion_tests = [
        "I absolutely adore this book!",
        "This product is disappointing.",
        "The meeting was average."
    ]
    
    print("감정 분류 테스트:")
    for text in emotion_tests:
        formatted = emotion_prompt.format(text=text)
        print(f"\n📝 테스트 텍스트: {text}")
        print("선택된 예시와 프롬프트:")
        print(formatted)
        print("-" * 40)
        
except Exception as e:
    print(f"⚠️ 감정 분류 예제 중 오류: {e}")

# =============================================================================
# 핵심 포인트 및 활용 팁
# =============================================================================
print("\n" + "=" * 60)
print("핵심 포인트 및 활용 팁")
print("=" * 60)

print("""
SemanticSimilarityExampleSelector의 장점:
1. 자동 예시 선택: 입력과 가장 관련 있는 예시만 선택
2. 확장성: 대량의 예시 중에서 효율적 선택
3. 적응성: 새로운 예시 동적 추가 가능
4. 비용 효율성: 불필요한 예시로 인한 토큰 낭비 방지

실제 활용 분야:
- 다국어 번역 시스템
- 코드 생성 및 변환
- 고객 서비스 응답 분류
- 창의적 글쓰기 도우미

주의사항:
- 임베딩 모델의 품질이 선택 품질을 좌우
- 초기 예시의 다양성과 품질이 중요
- 벡터스토어 설정과 유지관리 필요
- API 키와 토큰 관리 필수

성능 최적화 팁:
- k 값 조정으로 선택할 예시 개수 최적화
- 도메인별 임베딩 모델 고려
- 정기적인 예시 업데이트와 정제
- 벡터스토어 백업 및 버전 관리
""")

print("\n🎉 7.2.3 SemanticSimilarityExampleSelector 예제 완료!")

필요한 API 키들을 설정해주세요:
✅ 기존 OpenAI API 키를 사용합니다.

📝 Activeloop 토큰은 선택사항입니다 (로컬에서만 실행할 경우 생략 가능)


Activeloop 토큰 (선택사항, 엔터로 건너뛰기):  ········


Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


⏭️ Activeloop 토큰을 건너뛰었습니다 (로컬 저장소 사용)

7.2.3 의미적 유사성 기반 예시 선택자

원서 코드의 주요 수정사항:
1. 라이브러리 버전 호환성 문제 해결
2. SemanticSimilarityExampleSelector 생성 방식 변경
3. vectorstore 객체 직접 전달 방식으로 변경
4. 예시 삽입 방법 개선

핵심 개념 - SemanticSimilarityExampleSelector:
- 입력과 의미적으로 가장 유사한 예시를 자동 선택
- 벡터 임베딩을 사용한 유사도 계산
- 대량의 예시 중에서 효율적으로 관련성 높은 예시만 선택

사용할 예시 개수: 5
1. 0°C → 32°F
2. 10°C → 50°F
3. 20°C → 68°F
4. 30°C → 86°F
5. 40°C → 104°F

단계 1: 벡터 임베딩 및 벡터스토어 설정
Deep Lake Dataset in ./deeplake_temperature already exists, loading from the storage
✅ DeepLake 벡터스토어가 생성되었습니다

단계 2: SemanticSimilarityExampleSelector 생성
✅ SemanticSimilarityExampleSelector가 생성되었습니다

단계 3: 예시들을 벡터스토어에 삽입


Creating 5 embeddings in 1 batches of size 5:: 100%|██████████| 1/1 [00:00<00:00,  1.84it/s]


...

Dataset(path='./deeplake_temperature', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (12, 1536)  float32   None   
    id        text      (12, 1)      str     None   
 metadata     json      (12, 1)      str     None   
   text       text      (12, 1)      str     None   
✅ 새로운 예시가 추가되었습니다

재테스트: 45°C (새로 추가된 50°C 예시와 가장 유사할 것으로 예상)


Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


선택된 프롬프트:
Convert the temperature from Celsius to Fahrenheit.
(섭씨에서 화씨로 온도를 변환해줘.)

Input: 40°C
Output: 104°F

Input: 45°C
Output:

다른 도메인 예제: 감정 분류
Deep Lake Dataset in ./deeplake_emotion already exists, loading from the storage


Creating 6 embeddings in 1 batches of size 6:: 100%|██████████| 1/1 [00:00<00:00,  1.99it/s]


Dataset(path='./deeplake_emotion', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (12, 1536)  float32   None   
    id        text      (12, 1)      str     None   
 metadata     json      (12, 1)      str     None   
   text       text      (12, 1)      str     None   
감정 분류 테스트:

📝 테스트 텍스트: I absolutely adore this book!
선택된 예시와 프롬프트:
Classify the emotion of the given text as positive, negative, or neutral.
(주어진 텍스트의 감정을 긍정, 부정, 중립으로 분류하세요.)

Text: I love this movie!
Emotion: positive

Text: I love this movie!
Emotion: positive

Text: I absolutely adore this book!
Emotion:
----------------------------------------

📝 테스트 텍스트: This product is disappointing.
선택된 예시와 프롬프트:
Classify the emotion of the given text as positive, negative, or neutral.
(주어진 텍스트의 감정을 긍정, 부정, 중립으로 분류하세요.)

Text: This is terrible.
Emotion: negative

Text: This is terrible.
Emotion: n